In [1]:
import os
from dotenv import load_dotenv
import asyncio

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import ExternalTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.models.ollama import OllamaChatCompletionClient

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

# Create an OpenAI model client.
# model_client = OpenAIChatCompletionClient(
#     model="gpt-4o",
#     api_key=api_key
# )
model_client = OllamaChatCompletionClient(
    model="llama3.2"
)

# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    system_message="Provide constructive feedback. Respond with 'APPROVE' to when your feedbacks are addressed.",
)

# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("APPROVE")

# Create a team with the primary and critic agents.
team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=text_termination)

In [2]:
result = await team.run(task="Write a short poem about the fall season.")
print(result)

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=41, completion_tokens=108), metadata={}, content="As autumn leaves begin to fade,\nGolden hues and crimson shade,\nThe air is crisp, the winds do blow,\nA season's change, for all to know.\n\nThe trees stand tall, their branches high,\nTheir leaves rustling, as they say goodbye,\nTo summer's warmth, and long warm days,\nAnd welcome in the autumn's gray.\n\nThe scent of woodsmoke fills the air,\nAs nature's final dance begins to share,\nA time for harvest, a time for rest,\nThe fall season brings its quiet nest.", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=160, completion_tokens=320), metadata={}, content='Your poem effectively captures the essence of the fall season, conveying a sense of change and transition. Here are 

### Observing a Team

In [3]:
await team.reset()  # Reset the team for a new task.
async for message in team.run_stream(task="Write a short poem about the fall season."):  # type: ignore
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

source='user' models_usage=None metadata={} content='Write a short poem about the fall season.' type='TextMessage'
source='primary' models_usage=RequestUsage(prompt_tokens=41, completion_tokens=104) metadata={} content="As autumn leaves begin to sway,\nGolden hues and crimson play,\nThe air is crisp, the wind is cold,\nA season's change, young and old.\n\nThe trees stand tall, their branches bare,\nTheir summer green now lost in air,\nThe earthy scent of fallen leaves,\nFills lungs with freshness that it weaves.\n\nThe sun sets early, soft and low,\nA fiery sky, a final glow,\nThe stars appear, like diamonds bright,\nA night of rest, before the winter's light." type='TextMessage'
source='critic' models_usage=RequestUsage(prompt_tokens=156, completion_tokens=213) metadata={} content='Here is some constructive feedback:\n\nYour poem effectively captures the essence of the fall season, evoking vivid imagery and sensory details that transport the reader to this time of year.\n\nSome sugges

In [4]:
await team.reset()  # Reset the team for a new task.
await Console(team.run_stream(task="Write a short poem about the fall season."))  # Stream the messages to the console.

---------- user ----------
Write a short poem about the fall season.
---------- primary ----------
As summer's warmth begins to fade,
The fall season starts its gentle shade,
Golden leaves and crimson bright,
A kaleidoscope of color in sight.

The air is crisp, the winds do blow,
And nature's final dance begins to show,
The trees stand tall, their branches high,
A fiery crown against the autumn sky.

The scent of woodsmoke wafts through the air,
As earthy aromas fill the atmosphere rare,
The stars appear, like diamonds bright,
On a clear night, in the fall's soft light.
---------- critic ----------
Here are some suggestions for improvement:

* Consider adding more sensory details to bring the poem to life. For example, what sounds do you hear during this season? What textures can be imagined?
* Some of the lines feel a bit generic, such as "A kaleidoscope of color in sight" and "The stars appear, like diamonds bright". Can you come up with more unique ways to describe these phenomena?


CancelledError: 

# Selector Group Chat

In [ ]:
from typing import Sequence

from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.messages import AgentEvent, ChatMessage
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient


In [ ]:
# Note: This example uses mock tools instead of real APIs for demonstration purposes
def search_web_tool(query: str) -> str:
    if "2006-2007" in query:
        return """Here are the total points scored by Miami Heat players in the 2006-2007 season:
        Udonis Haslem: 844 points
        Dwayne Wade: 1397 points
        James Posey: 550 points
        ...
        """
    elif "2007-2008" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2007-2008 is 214."
    elif "2008-2009" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2008-2009 is 398."
    return "No data found."


def percentage_change_tool(start: float, end: float) -> float:
    return ((end - start) / start) * 100


In [ ]:
model_client = OllamaChatCompletionClient(model="llama3.2")

planning_agent = AssistantAgent(
    "PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        WebSearchAgent: Searches for information
        DataAnalystAgent: Performs calculations

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
)

web_search_agent = AssistantAgent(
    "WebSearchAgent",
    description="An agent for searching information on the web.",
    tools=[search_web_tool],
    model_client=model_client,
    system_message="""
    You are a web search agent.
    Your only tool is search_tool - use it to find information.
    You make only one search call at a time.
    Once you have the results, you never do calculations based on them.
    """,
)

data_analyst_agent = AssistantAgent(
    "DataAnalystAgent",
    description="An agent for performing calculations.",
    model_client=model_client,
    tools=[percentage_change_tool],
    system_message="""
    You are a data analyst.
    Given the tasks you have been assigned, you should analyze the data and provide results using the tools provided.
    If you have not seen the data, ask for it.
    """,
)


In [ ]:
text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination

In [ ]:
selector_prompt = """Select an agent to perform task.

{roles}

Current conversation context:
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
Make sure the planner agent has assigned tasks before other agents start working.
Only select one agent.
"""


In [ ]:
team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,  # Allow an agent to speak multiple turns in a row.
)


In [ ]:
task = "Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"


In [ ]:
# Use asyncio.run(...) if you are running this in a script.
import asyncio

asyncio.run(Console(team.run_stream(task=task)))